In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk
from streamlit_option_menu import option_menu

model = pk.load(open('banking_RFmodel.pkl', 'rb'))
Age_encoder = pk.load(open('AGE_GROUP.pkl', 'rb')) 
CHILDREN_encoder = pk.load(open('CHILDREN_CATEGORY.pkl', 'rb')) 
GENDER_encoder = pk.load(open('label_encoder_CODE_GENDER.pkl', 'rb')) 
FLAG_OWN_CAR_encoder = pk.load(open('label_encoder_FLAG_OWN_CAR.pkl', 'rb')) 
FLAG_OWN_REALTY_encoder = pk.load(open('label_encoder_FLAG_OWN_REALTY.pkl', 'rb')) 
INCOME_TYPE_encoder = pk.load(open('label_encoder_NAME_INCOME_TYPE.pkl', 'rb')) 
EDUCATION_TYPE_encoder = pk.load(open('label_encoder_NAME_EDUCATION_TYPE.pkl', 'rb')) 
FAMILY_STATUS_encoder = pk.load(open('label_encoder_NAME_FAMILY_STATUS.pkl', 'rb')) 
HOUSING_TYPE_encoder = pk.load(open('label_encoder_NAME_HOUSING_TYPE.pkl', 'rb')) 
OCCUPATION_encoder = pk.load(open('label_encoder_OCCUPATION_TYPE.pkl', 'rb')) 
ORGANIZATION_encoder = pk.load(open('label_encoder_ORGANIZATION_TYPE.pkl', 'rb')) 

st.set_page_config(page_title= "Bank Loan Default Prediction", layout= "wide")
st.header('Bank Loan Default Prediction - Machine learning Model')

@st.cache_data
def load_loan_data():
    return pd.read_csv('loan_data_strmlit.csv')

@st.cache_data
def load_model_performance_data():
    return pd.read_csv('model_performance.csv')

@st.cache_data
def load_roc_auc_performance_data():
    return pd.read_csv('ROC_AUC_performance.csv')

loan_data = load_loan_data()
model_performance_data = load_model_performance_data()
ROC_AUC_performance_data = load_roc_auc_performance_data()

def table():
    with st.expander("Loan Database"):
        shwdata=st.multiselect("Filter Dataset", loan_data.columns, default= ["NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "OCCUPATION_TYPE","AGE_GROUP","CHILDREN_CATEGORY"])
        st.dataframe(loan_data[shwdata], use_container_width = True)

st.sidebar.image("bank-loan-illustration-download-in-svg-png-gif-file-formats--cash-finance-investment-banking-pack-business-illustrations-3322131.jpg", use_column_width= True)
with st.sidebar:
    menu = option_menu(
                menu_title = "Explore",
                options = ["Data", "Dashboard", "Prediction"],
                styles = {
                    "nav-link-selected": {"background-color": "#FF4B4B"}
                }
    )

if menu=="Data":
    table()
    st.markdown("""---""")
    st.subheader("Model Performance")
    st.dataframe(model_performance_data)
    st.dataframe(ROC_AUC_performance_data)
    st.markdown("""---""")
    st.subheader("Therefore the selected model is RANDOM FOREST CLASSIFIER")


if menu=="Dashboard":
    st.subheader("Bank Loan Data Visualizations")

    with st.container():
        col1, col2, col3 = st.columns(3)


        with col1:
            fig1, ax = plt.subplots(figsize=(12, 10))  # Adjust figsize as needed for columns
            ax.pie(loan_data.TARGET.value_counts(), autopct='%1.1f%%', labels=["Non-Defaulters", "Defaulters"])
            ax.set_title('Target Distribution', fontsize=20) #Added fontsize
            st.pyplot(fig1)

        with col2:
            fig2, ax = plt.subplots(figsize=(6, 5))
            ax.pie(loan_data.FLAG_OWN_CAR.value_counts(), autopct='%1.1f%%', colors=("#83D12F", "#F07534"), labels=["YES", "NO"])
            ax.set_title("Car Ownership", fontsize=10) #Added fontsize
            st.pyplot(fig2)

        with col3:
            fig3, ax = plt.subplots(figsize=(6, 5))
            ax.pie(loan_data.FLAG_OWN_REALTY.value_counts(), autopct='%1.1f%%', colors=("#F08784", "#A1FB8E"), labels=["YES", "NO"])
            ax.set_title('Realty Ownership', fontsize=10) #Added fontsize
            st.pyplot(fig3)


    with st.container():
        col4, col5 = st.columns(2) # Create 2 columns

        with col4:
            fig4, ax = plt.subplots(figsize=(10, 8))  # Adjust figsize for columns
            sns.countplot(x='NAME_INCOME_TYPE', data=loan_data, palette='Spectral', order=loan_data['NAME_INCOME_TYPE'].value_counts().index, ax=ax)
            ax.set_title('Income Distribution', fontsize=10) #Added fontsize
            ax.set_xlabel('Income Type', fontsize=8) #Added fontsize
            ax.set_ylabel('Count', fontsize=8) #Added fontsize
            ax.tick_params(axis='x', labelsize=8)  # Adjust label size
            ax.tick_params(axis='y', labelsize=8)
            st.pyplot(fig4)

        with col5:
            education = loan_data['NAME_EDUCATION_TYPE'].value_counts()
            fig5, ax = plt.subplots(figsize=(8, 7))  # Adjusted figsize
            sns.barplot(x=education.index, y=education.values, palette='viridis', ax=ax)
            ax.set_title('Education Distribution', fontsize=10) #Added fontsize
            ax.set_xlabel('Education', fontsize=8) #Added fontsize
            ax.set_ylabel('No. of Applicants', fontsize=8) #Added fontsize
            ax.tick_params(axis='x', rotation=45, labelsize=6)  # Adjust label size and rotation
            ax.tick_params(axis='y', labelsize=6)
            plt.tight_layout()
            st.pyplot(fig5)

    with st.container():
        col6 = st.columns(1)
        with col6[0]:
            fig6, ax = plt.subplots(figsize=(6, 4))
            occupation_counts = loan_data['OCCUPATION_TYPE'].value_counts()
            sns.barplot(x=occupation_counts.values, y=occupation_counts.index, palette='YlOrRd_r', ax=ax)
            ax.set_title('Occupation Distribution', fontsize=10) #Added fontsize
            ax.set_xlabel('Occupation Counts', fontsize=8) #Added fontsize
            ax.set_ylabel('Occupation', fontsize=8) #Added fontsize
            ax.tick_params(axis='x', labelsize=6)  # Adjust label size
            ax.tick_params(axis='y', labelsize=6)
            st.pyplot(fig6)


if menu=="Prediction":
    st.subheader("Bank Loan Defaulters Prediction")
    Customer_Gender = st.selectbox("Select Customer Gender", loan_data["CODE_GENDER"].unique())
    Cust_OWN_CAR = st.selectbox("Do Customer Own Car?", loan_data["FLAG_OWN_CAR"].unique())
    Cust_OWN_REALTY = st.selectbox("Do Customer Own House?", loan_data["FLAG_OWN_REALTY"].unique())
    Cust_INCOME_TYPE = st.selectbox("Select Customer Income Type", loan_data["NAME_INCOME_TYPE"].unique())
    Cust_EDUCATION_TYPE = st.selectbox("Select Customer Education Type", loan_data["NAME_EDUCATION_TYPE"].unique())
    Cust_FAMILY_STATUS = st.selectbox("Select Customer Family Status", loan_data["NAME_FAMILY_STATUS"].unique())
    Cust_HOUSING_TYPE = st.selectbox("Select Customer Housing type", loan_data["NAME_HOUSING_TYPE"].unique())
    Cust_OCCUPATION_TYPE = st.selectbox("Select Customer Occupation Type", loan_data["OCCUPATION_TYPE"].unique())
    Cust_ORGANIZATION_TYPE = st.selectbox("Select Customer Organisation Type", loan_data["ORGANIZATION_TYPE"].unique())
    Cust_AGE = st.selectbox("Select Customer Age Group", loan_data["AGE_GROUP"].unique())
    Children = st.selectbox("Children", loan_data["CHILDREN_CATEGORY"].unique())
    Cust_Total_Income = st.number_input("Enter your Total Income Amount:")
    Cust_loan_Amount = st.slider("Loan Amount", 40000, 5000000)


    if st.button("Predict"):
        encoded_gender_input = GENDER_encoder.transform([Customer_Gender])[0]  # Keep [0] to get the single encoded value
        encoded_car_input = FLAG_OWN_CAR_encoder.transform([Cust_OWN_CAR])[0]
        encoded_reality_input = FLAG_OWN_REALTY_encoder.transform([Cust_OWN_REALTY])[0]
        encoded_income_type_input = INCOME_TYPE_encoder.transform([Cust_INCOME_TYPE])[0]
        encoded_edu_type_input = EDUCATION_TYPE_encoder.transform([Cust_EDUCATION_TYPE])[0]
        encoded_familystatus_input = FAMILY_STATUS_encoder.transform([Cust_FAMILY_STATUS])[0]
        encoded_housing_type_input = HOUSING_TYPE_encoder.transform([Cust_HOUSING_TYPE])[0]
        encoded_occupation_input = OCCUPATION_encoder.transform([Cust_OCCUPATION_TYPE])[0]
        encoded_org_input = ORGANIZATION_encoder.transform([Cust_ORGANIZATION_TYPE])[0]
        encoded_age_input = Age_encoder.transform([[Cust_AGE]])[0]   # Reshape to (1, 1) and then extract
        trans_income_input = np.log1p(Cust_Total_Income)
        encoded_child_input = CHILDREN_encoder.transform([[Children]])[0]
        encoded_child_input = encoded_child_input.astype(int)
        trans_loanamt_input = np.log1p(Cust_loan_Amount)
    
        user_df = pd.DataFrame([[
            encoded_gender_input, encoded_car_input, encoded_reality_input, encoded_income_type_input, encoded_edu_type_input,
            encoded_familystatus_input, encoded_housing_type_input, encoded_occupation_input, encoded_org_input, trans_income_input,
            trans_loanamt_input, encoded_age_input, encoded_child_input]],
                                columns = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                                           'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE',
                                           'AMT_INCOME_TOTAL_LOG', 'AMT_CREDIT_log', 'Age_group_encoded', 'CHILDREN_encoded'])
        st.write(user_df)
    
        prediction = model.predict(user_df)
        st.write(prediction)
        if prediction[0] == 1:
            st.warning("This customer is predicted to default on the loan.")
        else:
            st.success("This customer is predicted to not default on the loan.")

    